## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Namatanai,PG,-3.67,152.43,87.06,62,28,1.30,scattered clouds
1,1,Bluff,NZ,-46.60,168.33,54.00,79,72,11.99,broken clouds
2,2,Tautira,PF,-17.73,-149.15,82.40,74,75,16.11,shower rain
3,3,Jamestown,US,42.10,-79.24,41.00,100,90,4.92,overcast clouds
4,4,Saint-Philippe,RE,-21.36,55.77,77.00,61,0,6.93,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Namatanai,PG,-3.67,152.43,87.06,62,28,1.30,scattered clouds
2,2,Tautira,PF,-17.73,-149.15,82.40,74,75,16.11,shower rain
4,4,Saint-Philippe,RE,-21.36,55.77,77.00,61,0,6.93,clear sky
5,5,Atuona,PF,-9.80,-139.03,78.78,73,1,18.90,clear sky
8,8,Bandarbeyla,SO,9.49,50.81,76.69,73,5,4.07,clear sky
9,9,Luanda,AO,-8.84,13.23,75.20,94,75,3.36,broken clouds
10,10,Busselton,AU,-33.65,115.33,75.99,32,14,7.83,few clouds
18,18,Redcliffe,AU,-27.23,153.10,77.00,65,75,11.41,broken clouds
19,19,Rikitea,PF,-23.12,-134.97,75.42,84,100,15.50,overcast clouds
20,20,Harper,LR,4.38,-7.72,77.34,86,85,4.23,moderate rain


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                198
City                   198
Country                198
Lat                    198
Long                   198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                198
City                   198
Country                198
Lat                    198
Long                   198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Long"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
0,Namatanai,PG,87.06,scattered clouds,-3.67,152.43,
2,Tautira,PF,82.40,shower rain,-17.73,-149.15,
4,Saint-Philippe,RE,77.00,clear sky,-21.36,55.77,
5,Atuona,PF,78.78,clear sky,-9.80,-139.03,
8,Bandarbeyla,SO,76.69,clear sky,9.49,50.81,
9,Luanda,AO,75.20,broken clouds,-8.84,13.23,
10,Busselton,AU,75.99,few clouds,-33.65,115.33,
18,Redcliffe,AU,77.00,broken clouds,-27.23,153.10,
19,Rikitea,PF,75.42,overcast clouds,-23.12,-134.97,
20,Harper,LR,77.34,moderate rain,4.38,-7.72,


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Long"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df = hotel_df.replace("", np.NaN)
clean_hotel_df=hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
0,Namatanai,PG,87.06,scattered clouds,-3.67,152.43,
2,Tautira,PF,82.40,shower rain,-17.73,-149.15,
4,Saint-Philippe,RE,77.00,clear sky,-21.36,55.77,
5,Atuona,PF,78.78,clear sky,-9.80,-139.03,
8,Bandarbeyla,SO,76.69,clear sky,9.49,50.81,
9,Luanda,AO,75.20,broken clouds,-8.84,13.23,
10,Busselton,AU,75.99,few clouds,-33.65,115.33,
18,Redcliffe,AU,77.00,broken clouds,-27.23,153.10,
19,Rikitea,PF,75.42,overcast clouds,-23.12,-134.97,
20,Harper,LR,77.34,moderate rain,4.38,-7.72,


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Long"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Long"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))